In [1]:
%pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install audiomentations

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install numba --upgrade

  Using cached numba-0.56.4-cp39-cp39-win_amd64.whl (2.5 MB)
  Using cached llvmlite-0.39.1-cp39-cp39-win_amd64.whl (23.2 MB)
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.38.0
Note: you may need to restart the kernel to use updated packages.


ERROR: Cannot uninstall 'llvmlite'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [4]:
%pip install numpy==1.20

In [5]:
%pip install nemo_toolkit[all]

  Using cached nemo_toolkit-1.15.0-py3-none-any.whl (4.0 MB)
  Using cached onnx-1.13.1-cp39-cp39-win_amd64.whl (12.2 MB)
  Using cached numpy-1.24.2-cp39-cp39-win_amd64.whl (14.9 MB)
  Using cached sphinxcontrib_bibtex-2.5.0-py3-none-any.whl (39 kB)
  Using cached isort-4.3.21-py2.py3-none-any.whl (42 kB)
  Using cached editdistance-0.6.2-cp39-cp39-win_amd64.whl (22 kB)
  Using cached einops-0.6.0-py3-none-any.whl (41 kB)
  Using cached jiwer-2.5.1-py3-none-any.whl (15 kB)
  Using cached black-19.10b0-py36-none-any.whl (97 kB)
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached gradio-3.4.0-py3-none-any.whl (5.3 MB)
  Using cached inflect-6.0.2-py3-none-any.whl (34 kB)
  Using cached pytorch_lightning-1.8.6-py3-none-any.whl (800 kB)
  Using cached sacremoses-0.0.53-py3-none-any.whl
  Using cached sox-1.4.1-py2.py3-none-any.whl (39 kB)
  Using cached pyannote.core-5.0.0-py3-none-any.whl (58 kB)
  Using cached g2p_en-2.1.0-py3-none-any.whl (3.1 MB)
  Using cached 

ERROR: Cannot uninstall 'llvmlite'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


Загрузим наш аудио файл:

In [8]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams ['figure.figsize'] = [10, 7]

audio_path = "./1.wav"

audio, sr = librosa.load(audio_path, sr=None)

ImportError: Numba needs NumPy 1.21 or less

Визуализируем загруженный файл:

In [ ]:
wave_plot_basic = librosa.display.waveshow(audio, sr=sr)
plt.show()

Для отрисовки спектрограммы проведем предобработку сигнала:

In [ ]:
D = librosa.amplitude_to_db(np.abs(librosa.stft(audio)), ref=np.max)

Спектрограмма:

In [ ]:
spectrogram_basic = librosa.display.specshow(D, sr=sr, y_axis='linear', x_axis='time')

mel-спектрограмма:

In [ ]:
S = librosa.feature.melspectrogram(S=D, sr=sr)
mel_plt_basic = librosa.display.specshow(S, sr=sr, y_axis='mel', x_axis='time')

mfccs-спектрограмма:

In [ ]:
mfccs = librosa.feature.mfcc(S=S, sr=sr)
mfccs_plt_basic = librosa.display.specshow(mfccs, sr=sr, x_axis='time')

Добавим к нашему файлу аугментацию:

In [ ]:
from audiomentations import Compose, AddGaussianNoise, Gain

augment = Compose([AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=1.0),
                   Gain(min_gain_in_db=-10, max_gain_in_db=20, p=0.33)])

augmented_audio = augment(samples=audio, sample_rate=sr)


Визуализируем полученный результат:

In [ ]:
wave_plot_augmented = librosa.display.waveshow(augmented_audio, sr=sr)
plt.show()

In [ ]:
D_augmented = librosa.amplitude_to_db(np.abs(librosa.stft(augmented_audio)), ref=np.max)

In [ ]:
spectrogram_augmented = librosa.display.specshow(D_augmented, sr=sr, y_axis='linear', x_axis='time')

In [ ]:
S_augmented = librosa.feature.melspectrogram(S=D_augmented, sr=sr)
mel_plt_augmented = librosa.display.specshow(S_augmented, sr=sr, y_axis='mel', x_axis='time')

In [ ]:
mfccs_augmented = librosa.feature.mfcc(S=S_augmented, sr=sr)
mfccs_plt_augmented = librosa.display.specshow(mfccs_augmented, sr=sr, x_axis='time')

Наглядное сравнение спектрограмм файлов:

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(15,7))
librosa.display.waveshow(audio, sr=sr, ax=ax[0])
librosa.display.waveshow(augmented_audio, sr=sr, ax=ax[1])

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(15,7))
librosa.display.specshow(D, sr=sr, y_axis='linear', x_axis='time', ax=ax[0])
librosa.display.specshow(D_augmented, sr=sr, y_axis='linear', x_axis='time', ax=ax[1])

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(15,7))
librosa.display.specshow(S, sr=sr, y_axis='mel', x_axis='time', ax=ax[0])
librosa.display.specshow(S_augmented, sr=sr, y_axis='mel', x_axis='time', ax=ax[1])

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(15,7))
librosa.display.specshow(mfccs, sr=sr, x_axis='time', ax=ax[0])
librosa.display.specshow(mfccs_augmented, sr=sr, x_axis='time', ax=ax[1])

Распознавание речи на базе NeMo:

In [ ]:
import nemo.collections.asr as nemo_asr
model = nemo_asr.models.ASRModel.from_pretrained(model_name="QuartzNet15x5Base-En")
